<a href="https://colab.research.google.com/github/haleelsada/Cryptweets/blob/main/Cryptweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cryptweet**

**Integrating Twitter API with huggingface transformers to predict future price of a cryptocurrency**

In [22]:
!pip -q install transformers

import tweepy
from transformers import pipeline
import json
import pandas,numpy

In [5]:
from tweepy.auth import OAuthHandler

auth = OAuthHandler('t6d8uj4w9P6DjrNSmfSd42gEI', 'ZLcVXunCbN6NO4rUup6vTR33rO32epm0LFHLkzFLZnhjbQmQzZ')
auth.set_access_token('1405483945379074054-ktOMmQ6HUcZwzOxiHfjcFmP74hkTn5', 'hV792qfEAuJcvZmkzmoMf61qaaSnoV8D2YiIFYjzAgr06')


api = tweepy.API(auth,wait_on_rate_limit=True)

In [50]:
tweets=tweepy.Cursor(api.search, "giveaway",result_type='popular',tweet_mode="extended",count=100).items(200)
j=0
tweet=[]
for _ in range(10):
  for i in tweepy.Cursor(api.search, "#ethereum",tweet_mode="extended",count=100).items(300):
    i = json.dumps(i._json)
    i = json.loads(i)
    if i['full_text'] not in tweet:
      tweet.append(i['full_text'])
len(tweet)


163

In [33]:
len(tweet)

139

In [34]:
for j in i:
  print(j,i[j])

created_at Sat Apr 16 13:52:55 +0000 2022
id 1515327378465173510
id_str 1515327378465173510
full_text RT @antcolonyonmars: #Crofam #Defi #Nft #web3gaming

Final few days to join the Migration to #Ethereum with @Billyracxx
  
Get your #Creatu…
truncated False
display_text_range [0, 140]
entities {'hashtags': [{'text': 'Crofam', 'indices': [21, 28]}, {'text': 'Defi', 'indices': [29, 34]}, {'text': 'Nft', 'indices': [35, 39]}, {'text': 'web3gaming', 'indices': [40, 51]}, {'text': 'Ethereum', 'indices': [93, 102]}], 'symbols': [], 'user_mentions': [{'screen_name': 'antcolonyonmars', 'name': 'pAiNTer', 'id': 1455559883915411462, 'id_str': '1455559883915411462', 'indices': [3, 19]}, {'screen_name': 'Billyracxx', 'name': 'CULT LEADER', 'id': 254841263, 'id_str': '254841263', 'indices': [108, 119]}], 'urls': []}
metadata {'iso_language_code': 'en', 'result_type': 'recent'}
source <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
in_reply_to_status_id None
in_re

In [10]:
dir(api)


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_list_members',
 '_lookup_friendships',
 '_pack_image',
 '_remove_list_members',
 '_send_direct_message',
 'add_list_member',
 'add_list_members',
 'api_root',
 'auth',
 'blocks',
 'blocks_ids',
 'cache',
 'cached_result',
 'compression',
 'configuration',
 'create_block',
 'create_favorite',
 'create_friendship',
 'create_list',
 'create_media_metadata',
 'create_mute',
 'create_saved_search',
 'destroy_block',
 'destroy_direct_message',
 'destroy_favorite',
 'destroy_friendship',
 'destroy_list',
 'destroy_mute',
 'destroy_saved_search',
 'destroy_status',
 'favorites',
 'followers',
 'followers_ids',
 'friends',
 

In [43]:
!pip3 install emoji
sentimentmodel=pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")



     |████████████████████████████████| 175 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=c57763218fd0c5815d55f4476ba9506597bae4e4f43b9cd2dc6c3cbd941a802c
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

In [48]:
data=['the movie was good,but could have been better ']
sentimentmodel(data)

[{'label': 'POS', 'score': 0.903214156627655}]